In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt, geometry, to_geojson
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
""" shp_points = gp.read_file(r'D:\毕设\SHPmaps\openstreetmap_gugong\points.shp', encoding="gb18030")
shp_points.to_excel('gugong_points.xls', encoding='utf-8')
shp_multipolygons = gp.read_file(r'D:\毕设\SHPmaps\openstreetmap_gugong\multipolygons.shp', encoding="gb18030")
shp_multipolygons = shp_multipolygons.dropna(axis=0, subset='other_tags')
shp_multipolygons.to_excel('gugong_multipolygons.xls', encoding='utf8') """

In [ ]:
df = pd.DataFrame(pd.read_excel(r'D:\毕设\mysms\gugong_multipolygons.xls'))
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry = 'geometry')
gdf.head()

In [ ]:
now_position_polygon = geometry.Polygon([(
              116.39142952526362,39.91443343162902),
            (116.39142952526362,39.91358846112482),
            (116.39358833591018,39.91358846112482),
            (116.39358833591018,39.91443343162902),
            (116.39142952526362,39.91443343162902)])
#筛选在当前范围内的建筑面
cnt = 0
inscope_tags = []  #保存范围内建筑面的名称
inscope_point = []  #保存范围内建筑面的矢量数据
""" res_geojson = {                   #构造返回GeoJSON字符串
    "type": "FeatureCollection",
    "features": [],
} """
for idx, row in gdf.iterrows():
    if now_position_polygon.intersects(row['geometry']):
        cnt += 1
        point = row['geometry'].representative_point()
        ##将type改为大写
        tmp = json.loads(to_geojson(row['geometry']))
        #坐标转换：WGS84->BD09
        print("before trans")
        print(tmp['coordinates'][0])
        for xy in tmp['coordinates'][0]:
            xy = wgs84_to_bd09(xy[0], xy[1])
        print("after trans")
        print(tmp['coordinates'][0])
        break
        single_geojson = {
            "type": "Feature",
            "geometry": tmp
        }
        #res_geojson['features'].append(single_geojson)
        inscope_tags.append(row['other_tags'])
        inscope_point.append(point)

""" #计算空间关系
traid = {
    'firstQuadrant':[],
    'secondQuadrant':[],
    'thirdQuadrant':[],
    'forthQuadrant':[]
}
for i in range(0, cnt - 1):
    ax = inscope_point[i].x
    ay = inscope_point[i].y
    for j in range(i + 1, cnt):
        bx = inscope_point[j].x
        by = inscope_point[j].y
        #计算方位关系
        dx = ax - bx
        dy = ay - by
        #以赤道上1°~111km为比例，当两地点经度或纬度坐标差在1e-5以下时认为在一条直线上
        if dx >= 0 and dy > 0:
            traid['firstQuadrant'].append([bx, by])
        elif dx >= 0 and dy < 0:
            traid['forthQuadrant'].append([bx, by])
        elif dx < 0 and dy <= 0:
            traid['thirdQuadrant'].append([bx, by])
        else:
            traid['secondQuadrant'].append([bx, by])

print(traid) """

In [ ]:
###读取geojson
beihua_json = gpd.read_file(r"D:\毕设\mysms\beihua.json")

In [ ]:
path = 'D:\毕设\mysms\gugong.json'
f = open(path,'r',encoding='utf-8')
beihua_json = json.load(f)
for everyfeature in beihua_json['features']:
    everyfeature['properties']['parent'] = '北京故宫博物院'
with open("D:\毕设\mysms\gugong_new.json","w", encoding='utf-8') as f: ## 设置'utf-8'编码
    f.write(json.dumps(beihua_json, ensure_ascii = False, indent = 4))

In [20]:
from pymongo import MongoClient
from shapely import Polygon
from changestep.SpatialRelationship import PointCalculate, PolygonCalculate
from LocationConvert import getWgs84xy
from functools import cmp_to_key
myclient = MongoClient('localhost',27017)
collection = myclient['GeoDB']['detailGeo']
results = collection.find({"properties.parent":'北京化工大学（朝阳校区）'}) # 返回的是一个可迭代对象，需要遍历获得具体的值
gdf = gpd.GeoDataFrame(list(results))
userPosition = [116.41542034513238, 39.96887918686956]
#gdf.drop(columns='_id', axis = 1, inplace = True)
for idx, row in gdf.iterrows():
    geo = row['geometry']['coordinates'][0]
    geo_polygon = Polygon(geo)
    row['geometry'] = geo_polygon
#创建当前可视范围
#now_position_polygon = geometry.Polygon([(116.41442581939029, 39.96786235267895), (116.41641676053597, 39.9678971434655), (116.41641481344503, 39.96989648128038), (116.41442387299895, 39.96986169064365)])
wgs84_scope = [(116.41442581939029, 39.96786235267895), (116.41641676053597, 39.9678971434655), (116.41641481344503, 39.96989648128038), (116.41442387299895, 39.96986169064365)]
#筛选在当前范围内的建筑面
cnt = 0
inscope_tags = []  #保存范围内建筑面的名称
inscope_polygon = []  #保存范围内建筑面的
now_position_point = []
res_geojson = {                   #构造返回GeoJSON字符串
    "type": "FeatureCollection",
    "features": [],
}
for idx, row in gdf.iterrows():
    if now_position_polygon.intersects(row['geometry']):
        cnt += 1
        polygon = row['geometry']#.representative_point()
        now_position_point.append(row['geometry'].representative_point())
        inscope_tags.append(row['properties']['name'])
        inscope_polygon.append(polygon)
directionalPattern = {'北方':[], '东北方':[], '东方':[], '东南方':[], '南方':[], '西南方':[], '西边':[], '西北边':[], '之中':[]}
for i in range(0, cnt):
    print('test test test')
    print(now_position_point[i].x, now_position_point[i].y)
    res_rela = PointCalculate([now_position_point[i].x, now_position_point[i].y], [userPosition[0], userPosition[1]])
    print(res_rela, inscope_tags[i])
    directionalPattern[res_rela].append({'idx': i, 'geometry': inscope_polygon[i], 'representPoint': now_position_point[i]})

mystr = ''
for key, value in directionalPattern.items():
    length = len(value)
    for i in range(length):
        if i != 0:
            mystr += '在' + inscope_tags[value[i - 1]['idx']] + '的' + PolygonCalculate(value[i]['geometry'], value[i - 1]['geometry'], wgs84_scope) + '是' + inscope_tags[value[i]['idx']] + '。'
        else:
            mystr += '\n我的' + key + '是' + inscope_tags[value[i]['idx']] + '。'
print(mystr)

#排序没有效果
""" def mysort(arr1, arr2):
    disa = abs(arr1['representPoint'].x - userPosition[0]) + abs(arr1['representPoint'].y - userPosition[1])
    disb = abs(arr2['representPoint'].x - userPosition[0]) + abs(arr2['representPoint'].y - userPosition[1])
    print(disa, disb)
    return disa < disb

print(directionalPattern)
for key, value in directionalPattern.items():
    value.sort(key = cmp_to_key(mysort)) """

test test test
116.41436876179978 39.96879255329412
北方 行政办公楼
test test test
116.41631212287552 39.968804646679104
东方 化学学院
test test test
116.41509582774808 39.97006223051996
西边 逸夫图书馆
test test test
116.41403093190115 39.96932616559867
西边 机电工程学院

我的北方是行政办公楼。
我的东方是化学学院。
我的西边是逸夫图书馆。在逸夫图书馆的西南方是机电工程学院。


" def mysort(arr1, arr2):\n    disa = abs(arr1['representPoint'].x - userPosition[0]) + abs(arr1['representPoint'].y - userPosition[1])\n    disb = abs(arr2['representPoint'].x - userPosition[0]) + abs(arr2['representPoint'].y - userPosition[1])\n    print(disa, disb)\n    return disa < disb\n\nprint(directionalPattern)\nfor key, value in directionalPattern.items():\n    value.sort(key = cmp_to_key(mysort)) "

'116.42811913748,39.976428277424,116.42953335104237,39.97784249098637,116.42929470798458,39.97804631141275,116.42902711847948,39.978210290472376,116.42873717146874,39.97833039045659,116.42843200641008,39.978403654105186,116.42811913748,39.978428277424,116.42780626854992,39.978403654105186,116.42750110349125,39.97833039045659,116.42721115648051,39.978210290472376,116.42694356697541,39.97804631141275,116.42670492391763,39.97784249098637,116.42811913748,39.976428277424'